<a href="https://colab.research.google.com/github/Jaehyeonee/2021_Cpp/blob/main/%EB%8D%B0%EB%A7%88%EA%B0%9C_%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C_1_%EC%A0%9C%EC%B6%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import torchvision.models as models
net = models.vgg16(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 115MB/s]


In [2]:
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets as dset


In [ ]:
# 시도1.
# 데이터 증강 (Data Augmentation)으로 학습 데이터를 증강하여 모델의 일반화 성능을 향상
# 이미지를 무작위로 회전, 이동, 반전, 크롭 , 정규화 변형을 적용
transf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [4]:
trainset = dset.CIFAR10(root='/data', train=True, download=True, transform=transf)
testset  = dset.CIFAR10(root='./data', train=False, download=True, transform=transf)

100%|██████████| 170498071/170498071 [00:02<00:00, 68246030.65it/s]


Extracting /data/cifar-10-python.tar.gz to /data


100%|██████████| 170498071/170498071 [00:02<00:00, 77074444.27it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [5]:
train_loader = DataLoader(trainset, batch_size=32)
test_loader = DataLoader(testset, batch_size=32)

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [6]:
net.classifier[6] = nn.Linear(in_features = 4096, out_features=10)
net.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
loss_func = nn.CrossEntropyLoss()
num_epoch = 10
learning_rate = 0.0001
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

In [8]:
for i in range(num_epoch):
  for j,[image,label] in enumerate(train_loader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = net.forward(x)
    loss = loss_func(output, y_)
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(f"Epoch: {i}/{num_epoch}, Loss: {loss.item()}")

    # if i % 10 == 0:
    #   print(loss)


Epoch: 0/10, Loss: 2.3679070472717285
Epoch: 0/10, Loss: 2.2869935035705566
Epoch: 0/10, Loss: 2.311293840408325
Epoch: 0/10, Loss: 2.2542130947113037
Epoch: 0/10, Loss: 2.136850595474243
Epoch: 0/10, Loss: 2.042180299758911
Epoch: 0/10, Loss: 2.045105218887329
Epoch: 0/10, Loss: 1.7632243633270264
Epoch: 0/10, Loss: 1.7878024578094482
Epoch: 0/10, Loss: 1.6726611852645874
Epoch: 0/10, Loss: 2.1472136974334717
Epoch: 0/10, Loss: 1.7280337810516357
Epoch: 0/10, Loss: 1.601696491241455
Epoch: 0/10, Loss: 1.204332709312439
Epoch: 0/10, Loss: 1.4726533889770508
Epoch: 0/10, Loss: 1.4743047952651978
Epoch: 0/10, Loss: 1.4385826587677002
Epoch: 0/10, Loss: 1.2950907945632935
Epoch: 0/10, Loss: 1.5095995664596558
Epoch: 0/10, Loss: 1.348199486732483
Epoch: 0/10, Loss: 1.2800581455230713
Epoch: 0/10, Loss: 1.5098172426223755
Epoch: 0/10, Loss: 1.4502227306365967
Epoch: 0/10, Loss: 1.4228332042694092
Epoch: 0/10, Loss: 1.2264087200164795
Epoch: 0/10, Loss: 1.3450149297714233
Epoch: 0/10, Loss: 

In [9]:
correct = 0
total = 0
with torch.no_grad():
  net.eval()
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Test accuracy: %.2f %%'%(100*correct/total))

Test accuracy: 90.97 %


///

In [ ]:
# 시도 2. 드롭아웃 레이어 추가
# 학습 과정에서 일부 뉴런을 랜덤하게 비활성화하여 과적합을 줄이는 효과를 가져오기 때문에
# 성능 향상을 할 때 좋은 효과를 냄.
net.classifier[2] = nn.Dropout(p=0.5)
net.classifier[5] = nn.Dropout(p=0.5)

net.to(device)


In [14]:
for i in range(num_epoch):
  for j,[image,label] in enumerate(train_loader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = net.forward(x)
    loss = loss_func(output, y_)
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(f"Epoch: {i}/{num_epoch}, Loss: {loss.item()}")

    # if i % 10 == 0:
    #   print(loss)


Epoch: 0/10, Loss: 2.4284090995788574
Epoch: 0/10, Loss: 2.2970457077026367
Epoch: 0/10, Loss: 2.376187562942505
Epoch: 0/10, Loss: 2.336583375930786
Epoch: 0/10, Loss: 2.341883659362793
Epoch: 0/10, Loss: 2.317565679550171
Epoch: 0/10, Loss: 2.233660936355591
Epoch: 0/10, Loss: 2.135793685913086
Epoch: 0/10, Loss: 2.2474286556243896
Epoch: 0/10, Loss: 2.119872570037842
Epoch: 0/10, Loss: 2.2700514793395996
Epoch: 0/10, Loss: 2.1721549034118652
Epoch: 0/10, Loss: 2.182450294494629
Epoch: 0/10, Loss: 2.1945436000823975
Epoch: 0/10, Loss: 2.3200154304504395
Epoch: 0/10, Loss: 2.0663976669311523
Epoch: 0/10, Loss: 2.0575809478759766
Epoch: 0/10, Loss: 1.9805070161819458
Epoch: 0/10, Loss: 1.905197262763977
Epoch: 0/10, Loss: 2.2507617473602295
Epoch: 0/10, Loss: 1.9998029470443726
Epoch: 0/10, Loss: 1.9884885549545288
Epoch: 0/10, Loss: 1.8969950675964355
Epoch: 0/10, Loss: 1.9382102489471436
Epoch: 0/10, Loss: 2.034087896347046
Epoch: 0/10, Loss: 1.997544765472412
Epoch: 0/10, Loss: 1.97

In [15]:
correct = 0
total = 0
with torch.no_grad():
  net.eval()
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Test accuracy: %.2f %%'%(100*correct/total))

Test accuracy: 91.33 %


드롭아웃 레이어 추가 후, 91.33% 상승

In [11]:
# 시도 3.
# VGG16 모델의 분류기 부분을 수정
num_features = net.classifier[6].in_features

hidden_size=512

# 기존 사전학습된 vgg16 모델에 새로운 은닉층 추가
hidden_layer = nn.Linear(num_features, hidden_size)
net.classifier[6] = hidden_layer

# 새로운 은닉층 이후에 분류를 위한 출력 레이어 추가
output_layer = nn.Linear(hidden_size, 10)
net.classifier.add_module("output", output_layer)

net.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [12]:
for i in range(num_epoch):
  for j,[image,label] in enumerate(train_loader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = net.forward(x)
    loss = loss_func(output, y_)
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(f"Epoch: {i}/{num_epoch}, Loss: {loss.item()}")

    # if i % 10 == 0:
    #   print(loss)


Epoch: 0/10, Loss: 2.362677812576294
Epoch: 0/10, Loss: 2.194993019104004
Epoch: 0/10, Loss: 2.189608573913574
Epoch: 0/10, Loss: 2.0450398921966553
Epoch: 0/10, Loss: 2.037992238998413
Epoch: 0/10, Loss: 1.8541847467422485
Epoch: 0/10, Loss: 1.7295056581497192
Epoch: 0/10, Loss: 1.5712298154830933
Epoch: 0/10, Loss: 1.4573496580123901
Epoch: 0/10, Loss: 1.1524579524993896
Epoch: 0/10, Loss: 1.2869796752929688
Epoch: 0/10, Loss: 1.0419543981552124
Epoch: 0/10, Loss: 1.0359219312667847
Epoch: 0/10, Loss: 0.8360897898674011
Epoch: 0/10, Loss: 0.7603418827056885
Epoch: 0/10, Loss: 0.5506576299667358
Epoch: 0/10, Loss: 0.46758928894996643
Epoch: 0/10, Loss: 0.3901715576648712
Epoch: 0/10, Loss: 0.26616349816322327
Epoch: 0/10, Loss: 0.38773199915885925
Epoch: 0/10, Loss: 0.36625590920448303
Epoch: 0/10, Loss: 0.33594653010368347
Epoch: 0/10, Loss: 0.565517783164978
Epoch: 0/10, Loss: 0.3048079013824463
Epoch: 0/10, Loss: 0.22525203227996826
Epoch: 0/10, Loss: 0.1064872145652771
Epoch: 0/10

In [13]:
correct = 0
total = 0
with torch.no_grad():
  net.eval()
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Test accuracy: %.2f %%'%(100*correct/total))

Test accuracy: 90.51 %


최종성능: 91.33% 모델 2. 사전훈련된 vgg16모델에서 드롭아웃 레이어를 2개 추가하여 가장 높은 성능 기록.

In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn

# 사전 학습된 VGG16 모델 불러오기
model = models.vgg16(pretrained=True)

# 드롭아웃 레이어 추가
model.classifier.add_module("dropout1", nn.Dropout(p=0.5))
model.classifier.add_module("dropout2", nn.Dropout(p=0.5))

# 이미지 변환을 위한 transformation 구성
transform = transforms.Compose([
    transforms.Resize(256),  # 이미지 사이즈 조정
    transforms.CenterCrop(224),  # 이미지 크롭
    transforms.RandomHorizontalFlip(),   # 이미지 반전
    transforms.ToTensor(),  # 텐서로 바꿔줌
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]) # 정규화 진행
])
# 모델과 변환을 하나의 딕셔너리로 저장
model_data = {
    'model': model,
    'transform': transform
}

# 모델 데이터 저장
torch.save(model_data, 'model.pt')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 76.0MB/s]
